# Classification Model for Utah

In [2]:
# packages used
import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt
#import pydot_ng as pydot # visualization of trees (among many others)
#%matplotlib inline

#from sklearn import tree
#from sklearn.ensemble import BaggingClassifier
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.ensemble import AdaBoostClassifier
#from sklearn.model_selection import train_test_split
#from sklearn.model_selection import cross_val_score

#from IPython.display import Image # displaying images files in jupyter
#from IPython.display import IFrame # displaying pdf file in jupyter

### Data Loading

In [3]:
# Load the data
data_dir = 'Data/'
Stores = pd.read_csv(data_dir + 'AllStoresUtah.csv')

In [4]:
Stores.head()

,RTL_STORE_CD,RTL_FIPS_COUNTY_CD,STATE_CD,RTL_FIPS_COUNTY_DSC,CUSTCOL_57,RTL_CITY_DSC,RTL_PREMISE_TYPE_CD,RTL_PREMISE_TYPE_DSC,RTL_CHANNEL_CD,RTL_CHANNEL_DSC,RTL_SUBCHANNEL_CD,RTL_SUBCHANNEL_DSC,RTL_BEER_FLAG,RTL_LIQUOR_FLG,WJXBFS1
0,100490116,035,UT,SALT LAKE,49035,SANDY,OFF,OFF PREMISE,05,GROCERY,5,SUPERMARKET-CONVENTIONAL,Y,N,107.2500
1,102146472,053,UT,WASHINGTON,49053,ST GEORGE,ON,ON PREMISE,50,DINING,2,CASUAL DINING,Y,Y,5.0000
2,100635631,005,UT,CACHE,49005,LOGAN,OFF,OFF PREMISE,07,CONVENIENCE STORE,7,CONVENTIONAL CONVENIENCE,Y,N,26.0000
3,101972646,011,UT,DAVIS,49011,LAYTON,ON,ON PREMISE,55,CATERER,3,BANQUET HALL,Y,Y,24.0000
4,100447936,049,UT,UTAH,49049,OREM,OFF,OFF PREMISE,07,CONVENIENCE STORE,7,CONVENTIONAL CONVENIENCE,Y,N,224.0835
